<a href="https://colab.research.google.com/github/BrenoAV/ChatBot_Pizzaria_Super_Legal/blob/main/ChatBot_Pizzaria_Super_Legal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot da Pizzaria Super Legal 🍕😊👍

ChatBot criado para competição da **Imersão de IA da Alura 2° Edição** com parceria com o Google utilizando o Gemini como IA Generativa.

**Autor:** BrenoAV

**Última de vez modificado:** 5/11/2024

# Sumário:

* 1. Instalação de depedências
* 2. Importando as principais bibliotecas
* 3. Documentos
* 4. Modelo Generativo
* 5. Exemplos
    *   5.1 Texto
    *   5.2 Audio

# 1. Instalação de depedências

In [1]:
!pip install -q -U google-generativeai

# 2. Importando as principais bibliotecas

A biblioteca Pytz e datetime serão usadas para obter o dia atual no Brasil e verificar se é um dia de promoção da **Pizzaria Super Legal** 🍕😊👍

In [2]:
from datetime import datetime
import pytz
import numpy as np
from numpy.linalg import norm
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
import textwrap

GEMINI_KEY = userdata.get("GEMINI_KEY")

# Configurando a API KEY usando o valor vindo do `Secrets`
genai.configure(api_key=GEMINI_KEY)

# Definindo o fuso horário para o Brasil da biblioteca datetime do Python
date = datetime.now(pytz.timezone('America/Recife'))

Código para listar todos os modelos de geração de conteúdo que poderão ser usados no ChatBot. Eu optei por escolher o `gemini-1.5-pro`

In [3]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


# 3. Documentos

Aqui vem os documentos referentes da pizzaria, nesse caso estou usando simplesmente o cardápio da pizzaria com informações de promoções. Poderiam ter mais documentos e fazer um 'embedding' deles para realizar uma pesquisa semântica (Aula 5) retornando, incialmente, os documentos com valor semântico próximo para depois ser iniciado o chatbot com esse contexto.

O cardápio é curto para não exceder o limite gratuito da API Gemini. Consiste de valores diferentes para pizzas tradicionais, pizzas especiais, pizzas doces e bebidas. Ao final, se tem uma promoção que a gestão colocou para certos dias da semana.

In [4]:
doc_menu = """Cardápio da pizzaria super legal com todos os valores de pizzas e bebidas disponíveis na pizzaria.

# Pizzas Tradicionais

Preços das pizzas tradicionais:
  - R$ 20,00 (Pequena - P)
  - R$ 30,00 (Média - M)
  - R$ 40,00 (Grande - G)

01. Mussarela - Molho, mussarela, tomate, orégano e azeitona.
02. Dois queijos - Molho, mussarela, catupiry, orégano e azeitona.
03. Frango - Molho, frango, mussarela, milho, orégano e azeitona.
04. Vegetariana - Molho, milho, ervilha, cebola, tomate, palmito, brócolis, mussarela, orégano e azeitona.
05. Nordestina - Molho, carne cebola, mussarela, orégano e azeitona.
06. Calabresa - Molho, calabresa, bacon, orégano e azeitona.
07. Portuguesa - Molho, presunto, cebola, ervilha, ovos, mussarela, orégano e azeitona.
08. Baiana - Molho, calabresa, ovos, pimenta, cebola, mussarela, orégano e azeitona.

# Pizzas Especiais

Preços das pizzas especiais:
  - R$ 25,00 (Pequena - P)
  - R$ 35,00 (Média - M)
  - R$ 45,00 (Grande - G)

01. Camarão - Molho, camarão, mussarela, catupiry, orégano e azeitona.
02. Peru - Molho, peito de peru, mussarela, catupiry, orégano e azeitona.
03. Brasileira - Molho, peito de peru, parmesão, creme cheese, mussarela, bacon, orégano e azeitona.

# Pizzas Doces

Preços das pizzas doces:
  - R$ 25,00 (Pequena - P)
  - R$ 35,00 (Média - M)
  - R$ 45,00 (Grande - G)

01. Sonho de Valsa - Bombons, creme de leite, leite condensado e cereja.
02. Brigadeiro - Chocolate, granulado, leite condensado e cereja.
03. Chocovalsa - Chocolate, bombons, leite condensado, creme de leite e cereja.

# Bebidas

01. Água (R$ 2,00)
02. Refrigerante Lata (R$ 5,00)
03. Refrigerante 1L (R$ 8,00)
04. Refrigerante 2L (R$ 12,00)
05. Suco 600 mL (R$ 5,00)

# Promoções das Pizzas

Promoção: Nas sexta-feiras, sábado e domingo as pizzas tradicionais de Frango e Calabresa estão com um desconto de 30% para todos os tamanhos

"""

# 4. Modelo Generativo (gemini-1.5-pro-latest)

O modelo utilizado será o `gemini-1.5-pro-latest`, porque a versão `gemini-1.0-pro` não estava aceitado o parâmetro `system_instruction` ainda. Como configurações da geração (`generation_config`) foi usado um `temperature` com um valor baixo de `0.1` para que o modelo seja preciso/conservador por se tratar de informar ao cliente o seu pedido final e valor da compra.

Como `system_instructions` foi informado ao modelo agir como um atendente de uma pizzaria, a data de hoje (maneira dinâmica) para possíveis promoções, não fornecer nada que esteja fora do escopo da pizzaria, o cliente precisa informar a pizza desejada com o seu tamanho para ser cobrado propriamente e quando finalizar o pedido informar na última linha `fim do pedido` que será usado para finalizar o chat.


In [22]:
generation_config = {
    "temperature": 0.1,
    "candidate_count": 1
}

gen_model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    system_instruction=["Aja como um antendente expert de uma pizzaria",
                        f"A data de hoje é {date.date().strftime('%A')}",
                        "Não forneça nenhuma informação que não esteja no contexto da pizzaria, por exemplo, inventar novos tamanhos de pizzas ou inventar novos sabores de pizza.",
                        "Aplique os possíveis descontos das promoções da pizzaria e informe na resposta final o que teve desconto (se aplicável).",
                        "Responda quebrando linhas de no máximo 80 caracters e discrimine todos os valores."
                        "Quando finalizar o pedido simplesmente retorne na última linha: fim do pedido"
                        ]
)


Foi criado uma função chamada `start_chatbot_pizzaria` que recebe o contexto da pizzaria (no nosso caso o cardápio) e inicia uma sessão de chat onde o cliente poderá fazer seu pedido e receber o valor a ser pago para dar continuidade ao processo.

Existem duas modalidades a serem usadas:
    
- texto
- audio (nesse caso será apenas usado o áudio sem histórico do chatbot)


In [25]:
from typing import Union

def start_chatbot_pizzaria(context: str,
                           modo: bool = "texto",
                           audio_diretorio: str = "") -> None:
    if modo == "texto":
        chatbot_pizzaria = gen_model.start_chat(history=[])

        pedido = input("Por favor, informe seu pedido (digite 'sair' para finalizar o antendimento)\nComando: ")
        fim_do_pedido = None

        while True:
            prompt = ("O contexto da pizzaria é:\n\n"
                    "---"
                    f"{context}\n\n"
                    "---"
                    f"Cliente: {pedido}\n"
                    )
            response = chatbot_pizzaria.send_message(prompt)
            print("ChatBot:\n\n", response.text)
            print("-"*60)
            fim_do_pedido = response.text.split("\n")[-2]
            if fim_do_pedido == "fim do pedido":
                break
            pedido = input("Comando: ")
            print("-"*60)
            if pedido.lower().strip() == "sair":
                break
    elif modo == "audio":

        audio_file = genai.upload_file(path=audio_diretorio)
        prompt = ("O contexto da pizzaria é:\n"
                f"{context}\n"
                "Forneça o resumo do pedido discriminado. Baseado no áudio.")

        response = gen_model.generate_content([prompt, audio_file])
        print("ChatBot:\n", response.text)




# 5. Exemplos

Segue alguns exemplos usando o modo texto e modo áudio do ChatBot

# 5.1 Textos

In [26]:
start_chatbot_pizzaria(context=doc_menu, modo="texto")

Por favor, informe seu pedido (digite 'sair' para finalizar o antendimento)
Comando: Boa noite! Gostaria de uma pizza grande portuguesa e uma pizza média de brigadeiro
ChatBot:

 Boa noite! Anotei a pizza grande portuguesa.
E a pizza média de brigadeiro.
Mais alguma coisa?

------------------------------------------------------------
Comando: Apenas isso
------------------------------------------------------------
ChatBot:

 Ok, uma pizza grande portuguesa e uma pizza média de brigadeiro.
O total é R$ 75,00.
fim do pedido

------------------------------------------------------------


In [27]:
start_chatbot_pizzaria(context=doc_menu, modo="texto")

Por favor, informe seu pedido (digite 'sair' para finalizar o antendimento)
Comando: Gostaria de uma pizza de calabresa e um refrigerante de 2L para viagem
ChatBot:

 Boa noite! Qual o tamanho da pizza? 
Pequena, média ou grande?

------------------------------------------------------------
Comando: média
------------------------------------------------------------
ChatBot:

 Certo, uma pizza de calabresa média e um refrigerante de 2L, correto?



------------------------------------------------------------
Comando: correto, apenas isso meu pedido
------------------------------------------------------------
ChatBot:

 Perfeito! O valor total do seu pedido é R$ 20,40, já 
incluso o desconto de 30% na pizza de calabresa.

Desconto de R$ 9,60 na pizza de calabresa.
fim do pedido

------------------------------------------------------------


## 5.2 Áudios

In [28]:
# Baixando os dois áudios de teste

!wget https://github.com/BrenoAV/ChatBot_Pizzaria_Super_Legal/raw/main/audios/pedido-1.m4a
!wget https://github.com/BrenoAV/ChatBot_Pizzaria_Super_Legal/raw/main/audios/pedido-2.m4a

--2024-05-11 10:22:04--  https://github.com/BrenoAV/ChatBot_Pizzaria_Super_Legal/raw/main/audios/pedido-1.m4a
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/BrenoAV/ChatBot_Pizzaria_Super_Legal/main/audios/pedido-1.m4a [following]
--2024-05-11 10:22:04--  https://raw.githubusercontent.com/BrenoAV/ChatBot_Pizzaria_Super_Legal/main/audios/pedido-1.m4a
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120803 (118K) [audio/mp4]
Saving to: ‘pedido-1.m4a’

pedido-1.m4a        100%[===================>] 117.97K  --.-KB/s    in 0.004s  

2024-05-11 10:22:04 (31.4 MB/s) - ‘pedido-1.m4a’ saved [120803/1208

In [29]:
start_chatbot_pizzaria(context=doc_menu,
                       modo="audio",
                       audio_diretorio="pedido-1.m4a")

ChatBot:
 Boa noite! Uma pizza Calabresa Grande para viagem.
Hoje temos a promoção de 30% de desconto na Calabresa Grande.
O valor da pizza com desconto é R$ 28,00.
fim do pedido



In [30]:
start_chatbot_pizzaria(context=doc_menu,
                       modo="audio",
                       audio_diretorio="pedido-2.m4a")

ChatBot:
 Olá! Tudo bem?

Seu pedido é:
- Pizza Calabresa Média
- Refrigerante 2 Litros

O valor da Pizza Calabresa Média com desconto de 30% é R$ 21,00
O valor do Refrigerante 2 Litros é R$ 12,00

Total do pedido R$ 33,00

fim do pedido

